In [1]:
using Gurobi, LinearAlgebra, Ipopt, CSV, JuMP, DataFrames, Clp

### Model 1: Maximum coverage with a limit of areas that can be covered, x and y not within .01 of existing electricity

In [76]:
model1 = Model(with_optimizer(Gurobi.Optimizer))

max_units = 25

df= CSV.read("data.csv", copycols=true)
electricity = CSV.read("electricity.csv", copycols=true)
fdf = filter(row -> row[:value] > .0025, df)
m = size(fdf)[1]
n = size(electricity)[1]

@variable(model1, b[1:m], Bin)

@constraint(model1,sum(b[i] for i=1:m)<=max_units)

for i=1:m
    for j=1:n
        @constraint(model1, abs(fdf[7][i] - electricity[2][j])*b[i] >= .00001)
        @constraint(model1, abs(fdf[8][i] - electricity[3][j])*b[i] >= .00001)
    end
end

@objective(model1, Max, sum(fdf[9][i]*b[i] for i = 1:m))

model1;

┌ Warning: `with_optimizer` is deprecated. Adapt the following example to update your code:
│ `with_optimizer(Ipopt.Optimizer)` becomes `Ipopt.Optimizer`.
│   caller = top-level scope at In[76]:1
└ @ Core In[76]:1
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = macro expansion at rewrite.jl:227 [inlined]
└ @ Core /Users/alandler/.julia/packages/MutableArithmetics/ZGFsK/src/rewrite.jl:227


Academic license - for non-commercial use only


┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = macro expansion at rewrite.jl:227 [inlined]
└ @ Core /Users/alandler/.julia/packages/MutableArithmetics/ZGFsK/src/rewrite.jl:227
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = macro expansion at rewrite.jl:227 [inlined]
└ @ Core /Users/alandler/.julia/packages/MutableArithmetics/ZGFsK/src/rewrite.jl:227
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = macro expansion at rewrite.jl:227 [inlined]
└ @ Core /Users/alandler/.julia/packages/MutableArithmetics/ZGFsK/src/rewrite.jl:227
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = macro expansion at rewrite.jl:227 [inlined]
└ @ Core /Users/alandler/.julia/packages/MutableArithmetics/ZGFsK/src/rewrite.jl:227


In [77]:
optimize!(model1)

Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (mac64)
Optimize a model with 554605 rows, 1227 columns and 555831 nonzeros
Model fingerprint: 0x32f8c3c6
Variable types: 0 continuous, 1227 integer (1227 binary)
Coefficient statistics:
  Matrix range     [8e-05, 3e+00]
  Objective range  [3e-03, 5e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-05, 2e+01]
Presolve time: 0.07s

Explored 0 nodes (0 simplex iterations) in 0.14 seconds
Thread count was 1 (of 8 available processors)

Solution count 0

Model is infeasible or unbounded
Best objective -, best bound -, gap -


In [78]:
objective_value(model1)

MathOptInterface.ResultIndexBoundsError{MathOptInterface.ObjectiveValue}: Result index of attribute MathOptInterface.ObjectiveValue(1) out of bounds. There are currently 0 solution(s) in the model.

In [79]:
soln = value.(b)
sol = [i for (i,n) in enumerate(soln) if n == 1]
print(sol)

MathOptInterface.ResultIndexBoundsError{MathOptInterface.VariablePrimal}: Result index of attribute MathOptInterface.VariablePrimal(1) out of bounds. There are currently 0 solution(s) in the model.

### Iteration 1: max_val = 25, filter .0025, distance .0001, 009, .045, 09, .111, 
3.907821227000001
[59, 374, 438, 483, 492, 511, 546, 547, 598, 599, 633, 634, 635, 650, 671, 673, 674, 713, 714, 928, 929, 973, 974, 1008, 1024]
###

### Model 2: Maximum summation of distance and coverage.

In [86]:
model2 = Model(with_optimizer(Gurobi.Optimizer))

df= CSV.read("data.csv", copycols=true)
max_units = 25
fdf = filter(row -> row[:value] > .0025, df)
m = size(fdf)[1]
n = size(electricity)[1]
electricity = CSV.read("electricity.csv", copycols=true)

@variable(model2, b[1:m], Bin)

@constraint(model2,sum(b[i] for i=1:m)<=max_units)

@objective(model2, Max, sum(fdf[9][i]*b[i]+abs(fdf[7][i] - electricity[2][j])+abs(fdf[8][i] - electricity[3][j]) for i = 1:m for j=1:n))

model2;

┌ Warning: `with_optimizer` is deprecated. Adapt the following example to update your code:
│ `with_optimizer(Ipopt.Optimizer)` becomes `Ipopt.Optimizer`.
│   caller = top-level scope at In[86]:1
└ @ Core In[86]:1
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at rewrite.jl:227
└ @ Core /Users/alandler/.julia/packages/MutableArithmetics/ZGFsK/src/rewrite.jl:227


Academic license - for non-commercial use only


┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at rewrite.jl:227
└ @ Core /Users/alandler/.julia/packages/MutableArithmetics/ZGFsK/src/rewrite.jl:227
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at rewrite.jl:227
└ @ Core /Users/alandler/.julia/packages/MutableArithmetics/ZGFsK/src/rewrite.jl:227
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at rewrite.jl:227
└ @ Core /Users/alandler/.julia/packages/MutableArithmetics/ZGFsK/src/rewrite.jl:227
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at rewrite.jl:227
└ @ Core /Users/alandler/.julia/packages/MutableArithmetics/ZGFsK/src/rewrite.jl:227


In [ ]:
optimize!(model2)
objective_value(model2)

In [84]:
arr = []

for i=1:m
    for j=1:size(electricity)[1]
        push!(arr,(minimum([abs(fdf[7][i] - electricity[2][j])])))
    end
end

sort(arr)

┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[84]:5
└ @ Core ./In[84]:5
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[84]:5
└ @ Core ./In[84]:5


277302-element Array{Any,1}:
 0.00011462176525256496
 0.00011462176525256496
 0.00011462176525256496
 0.00011462176525256496
 0.00011462176525256496
 0.00011462176525256496
 0.00011462176525256496
 0.00011462176525256496
 0.00011462176525256496
 0.00011462176525256496
 0.00011462176525256496
 0.00011462176525256496
 0.00011462176525256496
 ⋮                     
 2.9229891342632968    
 2.9229891342632968    
 2.9229891342632968    
 2.9230192670812905    
 2.9230192670812905    
 2.9230192670812905    
 2.923313142025359     
 2.923313142025359     
 2.923313142025359     
 2.92333660196363      
 2.92333660196363      
 2.92333660196363      